## Load data

In [27]:
file_test = 'C:\Data\Miroculus\A3_1_2016_text_inputs\A3_1_2016_binary_data_entities_normalize_rmstopwords_stem_test.tsv'
file_train = 'C:\Data\Miroculus\A3_1_2016_text_inputs\A3_1_2016_binary_data_entities_normalize_rmstopwords_stem_train.tsv'

In [28]:
import pandas as pd
test = pd.read_csv(file_test, quotechar='"', delimiter='\t', header=None, names=['Label', 'Text'])
train = pd.read_csv(file_train, quotechar='"', delimiter='\t', header=None, names=['Label', 'Text'])
test.head; train.head

<bound method NDFrame.head of              Label                                               Text
0      NO_RELATION  result identifi previous report breast cancer-...
1      NO_RELATION  conclus taken togeth mmvarentty1mm/ggvarentty1...
2         RELATION  revers correl mmvarentty1mm ggvarentty1gg also...
3         RELATION  result suggest recognit site mmvarentty1mm mmv...
4         RELATION  issu jci ghosh colleagu identifi uniqu microrn...
5      NO_RELATION  conclus mmvarentty1mm may promot cell ggvarent...
6      NO_RELATION        howev role mmvarentty1mm npc remain unknown
7         RELATION  targetscan predict ggvarentty1gg ggvarentty2gg...
8      NO_RELATION  increas mmvarentty1mm depend upon ggvarentty1g...
9      NO_RELATION  revers correl mmvarentty1mm ggvarentty1gg expr...
10        RELATION  studi identifi mmvarentty1mm mmvarentty2mm mic...
11        RELATION  show mmvarentty1mm oncosuppress microrna lost ...
12     NO_RELATION  newli identifi mmvarentty1mm/ggvarentty1

## Vectorise text data into numerical vector

### Using `CountVectorizer`

In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(analyzer='word', stop_words='english', ngram_range=(1,2), max_df=0.9, min_df=0.01)
train_tfidf_vectors = tfidf_vectorizer.fit_transform(train.Text)

** Note: ** Numbers not removed. Consider removing numbers.

`tfidf_vectorizer.vocabulary_` is a `dict`

In [30]:
tfidf_vectorizer.vocabulary_

{'001': 0,
 '05': 1,
 '16': 2,
 '3p': 3,
 '5p': 4,
 'act': 5,
 'activ': 6,
 'addit': 7,
 'affect': 8,
 'aim': 9,
 'alter': 10,
 'analys': 11,
 'analysi': 12,
 'analyz': 13,
 'apoptosi': 14,
 'arrest': 15,
 'assay': 16,
 'associ': 17,
 'bind': 18,
 'bioinformat': 19,
 'biomark': 20,
 'blot': 21,
 'breast': 22,
 'breast cancer': 23,
 'cancer': 24,
 'cancer cell': 25,
 'cancer tissu': 26,
 'carcinoma': 27,
 'caus': 28,
 'cell': 29,
 'cell cycl': 30,
 'cell growth': 31,
 'cell line': 32,
 'cell migrat': 33,
 'cell prolifer': 34,
 'chain': 35,
 'chain reaction': 36,
 'chang': 37,
 'clinic': 38,
 'cluster': 39,
 'colon': 40,
 'combin': 41,
 'compar': 42,
 'conclus': 43,
 'conclus mmvarentty1mm': 44,
 'confirm': 45,
 'contribut': 46,
 'control': 47,
 'correl': 48,
 'correl ggvarentty1gg': 49,
 'correl mmvarentty1mm': 50,
 'crc': 51,
 'cycl': 52,
 'data': 53,
 'decreas': 54,
 'decreas express': 55,
 'demonstr': 56,
 'demonstr mmvarentty1mm': 57,
 'depend': 58,
 'detect': 59,
 'determin': 60,
 

In [31]:
train_tfidf_vectors.shape

(10783, 341)

In [32]:
train_tfidf_vectors.data

array([ 0.31644033,  0.2707969 ,  0.19469512, ...,  0.40269706,
        0.26405972,  0.38915471])

In [33]:
print(train_tfidf_vectors.nnz) # number of non-zero elements
print(train_tfidf_vectors.nnz / float(train_tfidf_vectors.shape[0])) # non-zero per sample
print(train_tfidf_vectors.nnz / float(train_tfidf_vectors.shape[0] * train_tfidf_vectors.shape[1]) * 100) # non-zero per feature space in %

131234
12.170453491607159
3.5690479447528323


## Data Preprocessing: 
### Attributes and Target

### Convert categorical labels into numerical labels

In [34]:
from sklearn import preprocessing

In [35]:
le = preprocessing.LabelEncoder()
le.fit(train.Label)
list(le.classes_)

['NO_RELATION', 'RELATION']

In [36]:
y_train = le.transform(train.Label);
y_test = le.transform(test.Label);

print(y_train[0:5])
print(le.inverse_transform(y_train[0:5]))

[0 0 1 1 1]
['NO_RELATION' 'NO_RELATION' 'RELATION' 'RELATION' 'RELATION']


`X` is data, `y` is target. `X` has the size of `n_samples` x `n_features`, `y` has `n_samples` x 1 labels

In [37]:
X_train = train_tfidf_vectors

In [38]:
X_test = tfidf_vectorizer.transform(test.Text)

In [39]:
X_test.shape

(3595, 341)

In [40]:
X_test.data

array([ 0.22653549,  0.24207315,  0.27759155, ...,  0.17149076,
        0.34588406,  0.18785396])

## Log Res: 

In [41]:
from sklearn import linear_model
clf = linear_model.LogisticRegression(C=1e5, class_weight='balanced')
clf.fit(X_train, y_train)

LogisticRegression(C=100000.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [42]:
y_pred = clf.predict(X_test)

In [43]:
from sklearn import metrics
print(metrics.classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          0       0.74      0.73      0.73      1804
          1       0.73      0.74      0.73      1791

avg / total       0.73      0.73      0.73      3595



In [44]:
print(metrics.confusion_matrix(y_test, y_pred))

[[1308  496]
 [ 470 1321]]


### Coeficients of the decision function The top word

In [45]:
clf.coef_

array([[  7.00840247e-01,  -7.45100515e-02,   2.69920778e+00,
          5.07383131e-01,  -6.89658023e-01,   5.61005110e-01,
          1.10647200e+00,   8.67853840e-01,   4.06479961e-01,
         -5.82016373e-01,   1.11060139e+00,   3.88310953e-01,
         -6.18821438e-02,   2.74201815e-01,   3.29218847e-01,
          4.88445061e-03,  -1.74916636e-02,  -5.39687259e-01,
          1.02714209e+00,   1.75231418e+00,  -9.71141008e-01,
          3.46291298e+00,  -2.63556540e-01,   8.68203400e-01,
         -1.79536246e+00,   5.01395567e-01,   1.39641162e+00,
         -1.25788333e+00,   1.32858488e+00,   3.43593849e-01,
          2.24756728e+00,   2.22755760e-01,   2.56463000e+00,
          7.69328653e-01,   3.13943530e-01,   2.39345389e+00,
         -4.56638764e+00,   2.69924229e-01,  -9.66875999e-02,
          1.20398896e+00,   9.50453874e-01,   1.03955686e+00,
         -5.66913475e-01,   4.35123093e-01,   7.40621359e-01,
          7.49021310e-01,   5.13744591e-01,   5.44088734e-01,
        

In [46]:
len(clf.coef_[0,])

341

Get the index of coef with the maximum value.

In [47]:
clf.coef_.argmax()

338

In [48]:
m = clf.coef_.argmax()
print(list(tfidf_vectorizer.vocabulary_.keys())[list(tfidf_vectorizer.vocabulary_.values()).index(m)])

western


Get the index of the highest 10 coef value.

**Note** `clf.coef_` is ndarray, see below:

In [49]:
clf.coef_[0]

array([  7.00840247e-01,  -7.45100515e-02,   2.69920778e+00,
         5.07383131e-01,  -6.89658023e-01,   5.61005110e-01,
         1.10647200e+00,   8.67853840e-01,   4.06479961e-01,
        -5.82016373e-01,   1.11060139e+00,   3.88310953e-01,
        -6.18821438e-02,   2.74201815e-01,   3.29218847e-01,
         4.88445061e-03,  -1.74916636e-02,  -5.39687259e-01,
         1.02714209e+00,   1.75231418e+00,  -9.71141008e-01,
         3.46291298e+00,  -2.63556540e-01,   8.68203400e-01,
        -1.79536246e+00,   5.01395567e-01,   1.39641162e+00,
        -1.25788333e+00,   1.32858488e+00,   3.43593849e-01,
         2.24756728e+00,   2.22755760e-01,   2.56463000e+00,
         7.69328653e-01,   3.13943530e-01,   2.39345389e+00,
        -4.56638764e+00,   2.69924229e-01,  -9.66875999e-02,
         1.20398896e+00,   9.50453874e-01,   1.03955686e+00,
        -5.66913475e-01,   4.35123093e-01,   7.40621359e-01,
         7.49021310e-01,   5.13744591e-01,   5.44088734e-01,
         1.32818286e-01,

In [50]:
print(clf.coef_[0][0])
print(clf.coef_[0][1])

0.7008402467
-0.0745100514554


In [51]:
import numpy as np
topN = np.argsort(clf.coef_[0])[::-1][:20]
print(topN)

[338 153  21 249 111  99  98 100 312 308 314 230 206  66   2  32 313 208
 273 108]


In [52]:
for v in topN:
    print(list(tfidf_vectorizer.vocabulary_.keys())[list(tfidf_vectorizer.vocabulary_.values()).index(v)])


western
invers
blot
prostat
ggvarentty2gg
gene mmvarentty1mm
gene ggvarentty1gg
ggvarentty1gg
target ggvarentty1gg
taken
target mmvarentty1mm
partial
mmvarentty3mm mmvarentty4mm
directli
16
cell line
target ggvarentty2gg
mmvarentty4mm mmvarentty5mm
repress
ggvarentty1gg protein
